In [1]:
import os
import pandas as pd
import torch

import importlib
import pickle
import re

base_path = os.getcwd()
data_path = os.path.join(base_path, "data")
question_path = os.path.join(data_path, "questions")
python_path = os.path.join(data_path, "python")

In [ ]:
def rename_files_in_folder(folder_path):
    files = os.listdir(folder_path)
    
    old_pattern = re.compile(r'p(\d+).py')
    
    for file_name in files:
        if file_name.endswith('.py'):
            match = old_pattern.match(file_name)
            if match:
                idx = int(match.group(1))
                
                for attempt in range(10): 
                    new_file_name = f"p{idx:03d}_{attempt}.py"
                    new_file_path = os.path.join(folder_path, new_file_name)
                    
                    if not os.path.exists(new_file_path):
                        old_file_path = os.path.join(folder_path, file_name)
                        os.rename(old_file_path, new_file_path)
                        print(f"Renamed {file_name} to {new_file_name}")
                        break

folder_path = "./data/Answer_GPT4"
rename_files_in_folder(folder_path)

In [2]:
import ProblemSetReader
importlib.reload(ProblemSetReader)
problem_set_reader = ProblemSetReader.ProblemSetReader()
df = pd.read_pickle("./data/euler_df.pkl")

In [39]:
import PassatKEvaluator
importlib.reload(PassatKEvaluator)

<module 'PassatKEvaluator' from 'd:\\Document\\Ucla-doc\\263\\project\\NLP_TermProjectCode\\PassatKEvaluator.py'>

In [85]:
evaluator = PassatKEvaluator.PassAtKEvaluator()
answer_path = "./data/Answer_GPT4"

evaluator.evaluate_pass_at_k(answer_path, df, k=1)

In [88]:
evaluator.evaluate_pass_at_k(answer_path, df, k=1)

{'pass_at_k_count': 23,
 'total_count': 56,
 'pass_at_k_ratio': 0.4107142857142857}